In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import lightning as L

In [3]:
from src.model.modeling_krop import LitKrop
from src.data.dataset import get_dacon_train_dataloader, get_dacon_dev_dataloader


In [ ]:
SEED=42
DATASET_NAME = 'jwengr/DACON-Korean-Review-Obfuscation'
BATCH_SIZE=2
BASE_MODEL_NAME='Qwen/Qwen3-0.6B'
LEARNING_RATE = 5e-5
TASK='jamo'
TRAIN_MAX_LENGTH=256
VALID_MAX_LENGTH=512

In [5]:
L.seed_everything(SEED)

Seed set to 42


42

In [ ]:
train_dl = get_dacon_train_dataloader(DATASET_NAME, batch_size=BATCH_SIZE)
dev_dl = get_dacon_dev_dataloader(DATASET_NAME, batch_size=BATCH_SIZE)

README.md:   0%|          | 0.00/514 [00:00<?, ?B/s]

c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jinwo\.cache\huggingface\hub\datasets--jwengr--DACON-Korean-Review-Obfuscation. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/377k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/383k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9011 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1126 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1126 [00:00<?, ? examples/s]

In [7]:
lit_krop = LitKrop(
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    task=TASK
)

In [8]:
trainer = L.Trainer(
    precision='16-mixed',
    max_epochs=10,
    enable_checkpointing=True
)

Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
trainer.validate(lit_krop, dev_dl)

You are using a CUDA device ('NVIDIA GeForce RTX 5070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       valid_loss            8.223331451416016
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'valid_loss': 8.223331451416016}]

In [ ]:
trainer.fit(lit_krop, train_dl, dev_dl)

Signature:
trainer.fit(
    model: 'pl.LightningModule',
    train_dataloaders: Union[Any, lightning.pytorch.core.datamodule.LightningDataModule, NoneType] = None,
    val_dataloaders: Optional[Any] = None,
    datamodule: Optional[lightning.pytorch.core.datamodule.LightningDataModule] = None,
    ckpt_path: Union[str, pathlib.Path, NoneType] = None,
) -> None
Docstring:
Runs the full optimization routine.

Args:
    model: Model to fit.

    train_dataloaders: An iterable or collection of iterables specifying training samples.
        Alternatively, a :class:`~lightning.pytorch.core.datamodule.LightningDataModule` that defines
        the :class:`~lightning.pytorch.core.hooks.DataHooks.train_dataloader` hook.

    val_dataloaders: An iterable or collection of iterables specifying validation samples.

    datamodule: A :class:`~lightning.pytorch.core.datamodule.LightningDataModule` that defines
        the :class:`~lightning.pytorch.core.hooks.DataHooks.train_dataloader` hook.

    ckp